# CAP Theorem

## What is CAP Theorem?

- in a distributed system, you can only have 2 out of these 3 properties:
    1. __Consistency__: all nodes see the same data at the same time
        - any writes made to one node will be reflected in other nodes when read
    2. __Availability__: every request receives a response
        - with the exception of failing nodes
        - data not guaranteed to be the most up-to-date
    3. __Partition Tolerance__: if part of a system fails, the whole system still continues to operate
- __IN ANY SYSTEM, PARTITION TOLERANCE IS A MUST!!!__
    * network failures will happen => your system must be able to operate in spite of it
    * that's 1/3 already chosen
    * therefore, the real choice is between consistency vs availability if a network partition occurs

## When to Choose Consistency:

* some systems require consistency, even at the cost of availability
* Ticket Booking Systems: two people shouldn't be able to book a flight for the same seat!
* E-commerce Inventory: you can't sell the same item to multiple people
* Financial Systems: imagine if the prices for stocks on Fidelity were not up-to-date for an order

## When to Choose Availability:

* __would it be catastrophic if users saw inconsistent data for a bit? if yes, then choose consistency!__
* majority of systems can tolerate some inconsistency and should prioritize availability
    - the system eventually becoming consistent is fine
    - might just be slightly delayed
* Social Media: user A updates their profile but user B might not see the changes right away
* Content Platforms (like Netflix): an update to a movie description might not be propagated right away and that's ok
* Review Sites (like Yelp): showing outdated info like restaurant hours isn't the end of the world

## CAP Theorem in System Design Interviews:

* CAP Theorem should be one of the first things we talk about in an interview
* essentially in an SD interview, we begin by:
    1. aligning on functional requirements (features)
    2. defining non-functional requirements (system qualities) => CAP Theorem comes into play here

### If you prioritize Consistency, keep these in mind:
* Distributed Transactions: if you have multiple data stores (e.g. cache and database), you must ensure that they are in sync
    - use two-phase commit protocols (2PC)
        * if you update one store, you must update another
        * if the update in one fails, you must abort the update
        * all-or-nothing!!!
    - this adds complexity and higher latency to a system but ensures strong consistency
* Single-Node Solutions: using a single database instance to avoid propagation issues
    - limits scalability but ensures consistency without too many challenges by having a single source of truth
* Technology Choices:
    - Traditional RDBMSs: PostgreSQL, MySQL
    - Google Spanner
    - DyanmoDB (in strong consistency mode)

#### If you prioritize Availability: keep these in mind:
* Multiple Replicas: adding more read replicas but asynchronously replicating information between them
    - this allows for reads to have high availability at the cost of serving stale data
* Change Data Capture (CDC): use a CDC to track changes in the primary database to then propagate those changes asynchronously to replicas, caches, and other systems
    - allows primary system to remain available while updates flow through the system eventually
* Technology Choices:
    - Cassandra
    - DynamoDB (in multiple availability zone configuration)
    - Redis clusters

## Advanced CAP Theorem Considerations

* for senior/staff+ discussions
* sometimes you need to have both availability and consistency but for different features
* e.g. Ticketmaster:
    - booking a seat at an event: strong consistency to prevent double-booking
    - viewing event details: prioritize availability (showing outdated event information is ok)

### Different Levels of Consistency:
* __when talking about Consistency, its mostly associated with _strong consistency___
    - but there's actually a spectrum of consistency
* Strong Consistency: all reads reflect most recent write
    - most expensive consistency model for performance
    - necessary for systems that need absolute accuracy, i.e. bank account balances
* Casual Consistency: related events appear in the same order to all users
    - logical ordering of dependent actions
    - e.g. ensure comment 2 replies to comment 1 and not the other way around
* Read-your-own-writes Consistency: users always see their own updates immediately while others might see an older version
    - common insocial media platforms
    - i.e. see your own profile updates right away
* Eventual Consistency: system will become consistent over time but may have temporary inconsistencies
    - most relaxed form
    - used in DNS where temporary inconsistency is acceptable
    - this is the default behavior of most distributed databases
    - __this is what we are choosing when we prioritize availability__